## AE-FLOW: Autoencoders with Normalizing Flows for Medical Images Anomaly Detection

## Training
Training the model requires the following arguments, below each argument you will find the default value, that which is used in the original paper, and in our reproduction experiment.

- loss_alpha
    - 0.5
- loss_beta
    - 0.9
- optim_lr
    - 2e-3 (1e-3 for the chest XRAY dataset)
- optim_momentum
    - 0.9
- optim_weight_decay
    - 10e-5 (0 for the chest XRAY dataset)
- dataset
    - choice out of: chest_xray, OCT2017 and btad
- subnet_architecture
    - choice out of conv_like and resnet_like
- epochs
    - 100

To train the model(s) of choice, you can use the following command:

``` python train.py --loss_alpha 0.5 --loss_beta 0.9 --optim_lr 2e-3 --optim_momentum 0.9 --optim_weight_decay 10e-5 --dataset chest_xray --subnet_architecture conv_like --epochs 100```

Without providing any arguments the model will train for 15 epochs using the standard parameters on the chest_xray dataset, using the conv_like subnet. 

In [5]:
!python train.py --loss_alpha 0.5 --loss_beta 0.9 --optim_lr 2e-3 --optim_momentum 0.9 --optim_weight_decay 10e-5 --dataset chest_xray --subnet_architecture conv_like --epochs 1

test


## Evaluation

Evaluating a model first requires loading in the data, we can do so using the `load()` function from `dataloader.py`. This function takes in three arguments: dataset, batch_size and num_workers, and outputs four PyTorch dataloaders: train_loader, train_complete, validation_loader, and test_loader. 

The train_complete loader may seem unfamiliar, this is because during our normal training procedure (using train_loader), we only use samples considered normal. The train_complete loader contains both abnormal and normal samples.

In [ ]:
from dataloader import load

train_loader, train_complete, val_loader, test_loader = load('chest_xray', batch_size=64, num_workers=4)

Once we load in the data, we can utilize the `eval_model()` function from `train.py`. This function takes in three arguments: epoch, model and data_loader. The epoch argument is exclusively used during training and can take any value during evaluation. 

The `eval_model()` function returns a dictionary containing various performance metrics such as AUC, ACC, SEN, SPE, and F1-score. 

## Pre-trained Models

To use the `eval_model()` function we need to specify a model. Below is a table including all pre-trained models included in this repository.

| model | subnet | dataset | epochs | loss_alpha | loss_beta | optim_lr | optim_momentum | optim_weight_decay |
| -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- |
| ae_flow | conv_like | btad | 100 | 0.5 | 0.9 | 0.002 | 0.9 | 0.0001 |
| ae_flow | resnet_like | btad | 100 | 0.5 | 0.9 | 0.002 | 0.9 | 0.0001 |
| ae_flow | conv_like | OCT2017 | 100 | 0.5 | 0.9 | 0.002 | 0.9 | 0.0001 |
| ae_flow | resnet_like | OCT2017 | 100 | 0.5 | 0.9 | 0.002 | 0.9 | 0.0001 |
| ae_flow | conv_like | chest_xray | 100 | 0.5 | 0.9 | 0.002 | 0.9 | 0.0001 |
| ae_flow | resnet_like | chest_xray | 100 | 0.5 | 0.9 | 0.002 | 0.9 | 0.0001 |

To load in the pretrained model weights, we also need to import the network. The network is found in `ae_flow_model.py` and contains the model class; `AE_Flow_Model`, this is where we need to pass the pretrained model into.

As an example, below we load in the ae_flow model with resnet_like subnet for the chest_xray dataset.

In [ ]:
import torch
from model.ae_flow_model import AE_Flow_Model

# Here we define the model we want to load in
model_params = {'model': 'ae_flow', 'subnet_arc': 'subnet', 'dataset': 'chest_xray', 'epochs': 100, 'loss_alpha': 0.5, 'loss_beta': 0.9, 'optim_lr': 0.001, 'optim_momentum': 0.9, 'optim_weight_decay': 0.0}

# Load in the model
model = AE_Flow_Model()

# Load the pretrained weights into the model
model.load_state_dict(torch.load(str(model_params)+'.pt'))

# Set the model into evaluation mode
model.eval()

Now that we have loaded in our data and pretrained model, we can use the `eval_model()` function.

In [ ]:
from train import eval_model

# Note that here we set the epoch argument to 0, but this can be any value during evaluation
test_results = eval_model(0, model, test_loader)

## Results

Now that we have retrieved the test set results, we can simply print the obtained performance metrics.

In [ ]:
for metric in test_results:
    print(f'{metric}: {test_results[metric]*100}%')

## Contributions
* Jan Athmer - Project implementation, debugging

* Pim Praat - Project implementation, debugging

* Andre de Brandt - Writing notebook/blogpost, debugging

* Farrukh Baratov - Writing notebook/blogpost, debugging

* Thijs Wijnheijmer - Writing notebook/blogpost, debugging